In [1]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [2]:
# DATA TEST 1
stocks = {
    "S1": {"width": 1219, "weight": 4395 },
    "S2": {"width": 1219, "weight": 9260},
    "S3": {"width": 1219, "weight": 3475},
    "S4": {"width": 1219, "weight": 8535},
    # "S5": {"width": 236, "weight": 1571},
}

finish = {
    "F1": {"width": 235, "need_cut": 800 },
    "F2": {"width": 147, "need_cut": 1308},
    "F3": {"width": 136, "need_cut": 1290},
    "F4": {"width": 68, "need_cut": 600},
    "F5": {"width": 60, "need_cut": 170},
    "F6": {"width": 85, "need_cut": 132},
    "F7": {"width": 57, "need_cut": 100},
    "F8": {"width": 92, "need_cut": 100}, 
    "F9": {"width": 57, "need_cut": 735}, 
}

In [3]:
## PARAMETER

# over cut bound
BOUND = 0.3
MIN_MARGIN = 8

## Problem with known naive patterns

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to fit the width of the Mother Coil or 




In [4]:
from codes.create_patterns import *

naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND, MIN_MARGIN)

# display(naive_patterns)

ap_upper_bound(naive_patterns,finish)

No feasible pattern was found for Stock S4 and FG F6
No feasible pattern was found for Stock S4 and FG F7
No feasible pattern was found for Stock S4 and FG F8


{'F1': 1,
 'F2': 4,
 'F3': 4,
 'F4': 4,
 'F5': 1,
 'F6': 0,
 'F7': 0,
 'F8': 0,
 'F9': 5}

In [25]:
def cut_stock(stocks, finish, Np, m):
    # Np=len(finish) # CAI NAY CO ANH HUONG GI
    pattern_bilinear_prob = m
    pattern_bilinear_prob.eval("reset data;")
    pattern_bilinear_prob.set["S"] = list(stocks.keys())
    pattern_bilinear_prob.set["F"] = list(finish.keys())
    pattern_bilinear_prob.set["P"] = list(range(Np)) # list number 

    # s = {p: naive_patterns[p]["stock"] for p in range(len(naive_patterns))} # stock allocated to naive patterns (0, n)
    # w = {p: stocks[s[p]]["weight"]/stocks[s[p]]["width"] for p in range(len(patterns))} # weight per unit according to each naive pattern

    wstocks = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
    # w.update(wstocks)
    pattern_bilinear_prob.param["wu"] = wstocks
    pattern_bilinear_prob.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    pattern_bilinear_prob.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}
    pattern_bilinear_prob.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    pattern_bilinear_prob.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

    a_upper_bound = { #upper bound of possible cut over all kind of stocks
        f: max([naive_patterns[i]['cuts'][f] for i,_ in enumerate(naive_patterns)])
        for f in finish.keys()
    }

    pattern_bilinear_prob.param["a_upper_bound"] = a_upper_bound
    pattern_bilinear_prob.get_output("solve;")

    return pattern_bilinear_prob


## Pattern Generation: Bilinear Model
Let binary variable $b_{sp}\in\mathbb{Z}_2$ denote the assignment of stock $s$ to pattern $p$, and let $P = 0, 1, \ldots, N_p-1$ index a list of patterns, $wu_{s}$ the weight unit per mm wight of the coil stock, which is the weight divided by the width. For sufficiently large $N_p$, an optimal solution to the stock cutting problem is given by the model


A solution is attempted using a mixed-integer nonlinear optimization (MINLO) solver.

### TEST 0

\begin{align}
\min \quad & \sum_{s\in S} b_{sp} l^S_s - \sum_{f\in F}a_{fp}l^F_f && \forall p\in P\\
\text{s.t.} \quad
& \sum_{s\in S}b_{sp} =  1 && \forall p\in P \\
& 0.96 * (\sum_{s\in S}b_{sp} l^S_s) \leq \sum_{f\in F}a_{fp}l^F_f \leq \ \sum_{s\in S}b_{sp} l^S_s -\text{margin}_{S} && \forall p\in P  \\
& \sum_{s \in S}b_{sp} wu_{s} *\sum_{f \in F}a_{fp} l^S_{f}  \leq d_f && \forall p\in P\\
& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P \\
& b_{sp} \in \{0,1\} && \forall s\in S,  \forall p\in P \\
\end{align}

In [9]:
##### test 0
# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern
# Find the patterns of stock that minimize the loss BY PATTERN
m = AMPL()
m.option["solver"] = SOLVER_MINLO
m.eval(
    """
        set S;
        set F;
        set P;

        # length stock
        param width_s{S};
        # length finished pieces
        param width_f{F};
        
        # WEIGHT STOCK
        param w{S};
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        param a_upper_bound{F};

        # how many F pieces are returned from pattern p
        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss{p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern{p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin{p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin{p in P}:
          sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

        subject to demand{f in F, p in P}:
          a[f,p] * width_f[f] * sum{s in S} b[s, p] * wu[s] <= f_upper_demand[f];
    """
)

### TEST 1

\begin{align}
\min \quad & \sum_{s\in S} b_{sp} l^S_s - \sum_{f\in F}a_{fp}l^F_f \\
\text{s.t.} \quad
& 0.96 (b_{sp} l^S_s) \leq \sum_{f\in F}a_{fp}l^F_f \leq \ b_{sp} l^S_s -\text{margin} &&  \forall s\in S ,\forall p\in P  \\
& wu_{s} \sum_{p\in P}a_{fp} l^F_{f}  \leq d_f && \forall f\in F\\
& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P \\
& b_{sp} \in \{0,1\} && \forall s\in S,  \forall p\in P \\
\end{align}

In [6]:
##### TEST 1

# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern
# Find the patterns of stock that minimize the loss BY PATTERN

m1 = AMPL()
m1.option["solver"] = SOLVER_MINLO
m1.eval(
    """
        set S;
        set F;
        set P;

        # length stock
        param width_s{S};
        # length finished pieces
        param width_f{F};
        
        # WEIGHT STOCK
        param w{S};
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        param a_upper_bound{F};

        # how many F pieces are returned from pattern p
        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss{p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern{p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin{p in P, s in S}:
          sum{f in F} a[f,p] * width_f[f] <= b[s,p] * width_s[s] - 8;

        subject to demand{f in F, p in P, s in S}:
          a[f,p] * width_f[f] * wu[s] <= f_upper_demand[f];
    """
)

In [24]:
Np=len(finish)
solved_prob = cut_stock(stocks, finish, Np, m)

# Retrieve the patterns
patterns = []
for p in range(len(finish)):
    a = {f: round(solved_prob.var["a"][f, p].value()) for f in finish.keys()}
    try:
        patterns.append(
            {
                "stock": [s for s in stocks.keys() if solved_prob.var["b"][s, p].value() >= 0][0],
                "cuts": a,
            }
        )
    except:
        print("Cant find optimized pattern")
        
patterns

[{'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3': 0,
   'F4': 0,
   'F5': 0,
   'F6': 0,
   'F7': 0,
   'F8': 0,
   'F9': 0}},
 {'stock': 'S1',
  'cuts': {'F1': 0,
   'F2': 0,
   'F3